# WaveBank
`WaveBank` is an in-process database for accessing seismic time-series data. Any directory structure containing ObsPy-readable waveforms can be used as the data source. `WaveBank` uses a simple indexing scheme and the [Hierarchical Data Format](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) to keep track of each `Trace` in the directory. Without `WaveBank` (or another similar program) applications have implement their own data organization/access logic which is tedious and clutters up application code. `WaveBank` provides a better way. 



## Load Example Data
This tutorial will demonstrate the use of `WaveBank` on  two different [obsplus datasets](../datasets/datasets.ipynb). 

The first dataset, [crandall canyon](https://en.wikipedia.org/wiki/Crandall_Canyon_Mine), only has event waveform files. The second only has continuous data from two TA stations. We start by loading these datasets, making a temporary copy, and getting a path to their waveform directories.

In [1]:
%%capture
import obsplus

# make sure datasets are downloaded and copy them to temporary
# directories to make sure no accidental changes are made
crandall_dataset = obsplus.load_dataset("crandall_test").copy_to()
ta_dataset = obsplus.load_dataset("ta_test").copy_to()

# get path to waveform directories
crandall_path = crandall_dataset.waveform_path
ta_path = ta_dataset.waveform_path

In [2]:
crandall_path

PosixPath('/tmp/tmpbki2unve/crandall_test/waveforms')

## Create a WaveBank object
To create a `WaveBank` instance simply pass the class a path to the waveform directory.

In [3]:
bank = obsplus.WaveBank(crandall_path)

Utilizing the `udpate_index` method on the bank ensures the index is up-to-date. This will iterate through all files that are timestamped later than the last time `update_index` was run.

Note: If the index has not yet been created or new files have been added, `update_index` needs to be called.

In [4]:
bank.update_index()

WaveBank(base_path=/tmp/tmpbki2unve/crandall_test/waveforms, index_path=/tmp/tmpbki2unve/crandall_test/waveforms/.index.h5)

## Using a custom index path

If you are working from a data directory that doesn't have write access, you can specify a custom location for the index path:

In [5]:
import tempfile
from pathlib import Path

index_path = Path(tempfile.mkdtemp()) / "index.h5"
cust_ind_bank = obsplus.WaveBank(crandall_path, index_path=index_path)
cust_ind_bank.update_index()

WaveBank(base_path=/tmp/tmpbki2unve/crandall_test/waveforms, index_path=/tmp/tmpyry362cs/index.h5)

## Get waveforms

The files can be retrieved from the directory with the `get_waveforms` method. This method has the same signature as the ObsPy client `get_waveform` methods so they can be used interchangeably:

In [6]:
import obspy

t1 = obspy.UTCDateTime("2007-08-06T01-44-48")
t2 = t1 + 60
st = bank.get_waveforms(starttime=t1, endtime=t2)

`WaveBank` can filter on channels, locations, stations, networks, etc. using linux style search strings or regex. 

In [7]:
st2 = bank.get_waveforms(network="UU", starttime=t1, endtime=t2)

# ensure only UU traces were returned
for tr in st2:
    assert tr.stats.network == "UU"


In [8]:
st = bank.get_waveforms(starttime=t1, endtime=t2, station="O1??", channel="BH[NE]")

# test returned traces
for tr in st:
    assert tr.stats.starttime >= t1 - 0.00001
    assert tr.stats.endtime <= t2 + 0.00001
    assert tr.stats.station.startswith("O1")
    assert tr.stats.channel.startswith("BH")
    assert tr.stats.channel[-1] in {"N", "E"}


WaveBank also has a `get_waveforms_bulk` method for efficiently retrieving a large number of streams. 

In [9]:
args = [  # in practice this list may contain hundreds or thousands of requests
    (
        "TA",
        "O15A",
        "",
        "BHZ",
        t1 - 5,
        t2 - 5,
    ),
    (
        "UU",
        "SRU",
        "",
        "HHZ",
        t1,
        t2,
    ),
]
st = bank.get_waveforms_bulk(args)

## Yield waveforms
The Bank class also provides a generator for iterating large amounts of continuous waveforms. The following example shows how to get streams of one hour duration with a minute of overlap between the slices. 

The first step is to create a bank on a dataset which has continuous data. The example below will use the TA dataset.

In [10]:
ta_bank = obsplus.WaveBank(ta_path)

In [11]:
# get a few hours of kemmerer data
ta_t1 = obspy.UTCDateTime("2007-02-15")
ta_t2 = obspy.UTCDateTime("2007-02-16")

for st in ta_bank.yield_waveforms(
    starttime=ta_t1, endtime=ta_t2, duration=3600, overlap=60
):
    pass

## Put waveforms
Files can be added to the bank by passing a stream or trace to the `bank.put_waveforms` method. `WaveBank` does not merge files so overlap in data may occur if care is not taken.

In [12]:
# show that no data for RJOB is in the bank
st = bank.get_waveforms(station="RJOB")

assert len(st) == 0


In [13]:
# add the default stream to the archive (which contains data for RJOB)
bank.put_waveforms(obspy.read())
st_out = bank.get_waveforms(station="RJOB")

# test output
assert len(st_out)
for tr in st_out:
    assert tr.stats.station == "RJOB"



## Availability
`WaveBank` can be used to get the availability of data. The outputs can either be a dataframe or as a list of tuples in the form of [(network, station, location, channel, min_starttime, max_endtime)]. 

In [14]:
# get a dataframe of availability by seed ids and timestamps
bank.get_availability_df(channel="BHE", station="[OR]*")

,network,station,location,channel,starttime,endtime
0,TA,O15A,,BHE,2007-08-06 01:44:38.825000,2007-08-07 21:43:51.124998
1,TA,O16A,,BHE,2007-08-06 01:44:38.825000,2007-08-07 21:43:51.125000
2,TA,O18A,,BHE,2007-08-06 01:44:38.824998,2007-08-07 21:43:51.125000
3,TA,R16A,,BHE,2007-08-07 02:04:54.500000,2007-08-07 21:43:51.125000
4,TA,R17A,,BHE,2007-08-06 01:44:38.825000,2007-08-07 21:43:51.125000


In [15]:
# get list of tuples of availability
bank.availability(channel="BHE", station="[OR]*")

[('TA',
  'O15A',
  '',
  'BHE',
  2007-08-06T01:44:38.825000Z,
  2007-08-07T21:43:51.124998Z),
 ('TA',
  'O16A',
  '',
  'BHE',
  2007-08-06T01:44:38.825000Z,
  2007-08-07T21:43:51.125000Z),
 ('TA',
  'O18A',
  '',
  'BHE',
  2007-08-06T01:44:38.824998Z,
  2007-08-07T21:43:51.125000Z),
 ('TA',
  'R16A',
  '',
  'BHE',
  2007-08-07T02:04:54.500000Z,
  2007-08-07T21:43:51.125000Z),
 ('TA',
  'R17A',
  '',
  'BHE',
  2007-08-06T01:44:38.825000Z,
  2007-08-07T21:43:51.125000Z)]

## Get Gaps and uptime
`WaveBank` can return a dataframe of missing data with the `get_gaps_df` method, and a dataframe of reliability statistics with the `get_uptime_df` method. These are useful for assessing the completeness of an archive of contiguous data.

In [16]:
bank.get_gaps_df(channel="BHE", station="O*").head()

,network,station,location,channel,starttime,endtime,sampling_period,path,gap_duration
0,TA,O15A,,BHE,2007-08-06 01:45:48.799998,2007-08-06 08:48:30.024998,0 days 00:00:00.025000,TA.O15A..BHE__20070806T014438Z__20070806T01454...,0 days 07:02:41.225000
1,TA,O15A,,BHE,2007-08-06 08:49:39.999998,2007-08-06 10:47:15.624998,0 days 00:00:00.025000,TA.O15A..BHE__20070806T084830Z__20070806T08494...,0 days 01:57:35.625000
2,TA,O15A,,BHE,2007-08-06 10:48:25.599998,2007-08-07 02:04:54.499998,0 days 00:00:00.025000,TA.O15A..BHE__20070806T104715Z__20070806T10482...,0 days 15:16:28.900000
3,TA,O15A,,BHE,2007-08-07 02:06:04.474998,2007-08-07 02:14:14.100000,0 days 00:00:00.025000,TA.O15A..BHE__20070807T020454Z__20070807T02060...,0 days 00:08:09.625002
4,TA,O15A,,BHE,2007-08-07 02:15:24.074998,2007-08-07 03:44:08.474998,0 days 00:00:00.025000,TA.O15A..BHE__20070807T021414Z__20070807T02152...,0 days 01:28:44.400000


In [17]:
ta_bank.get_uptime_df()

,network,station,location,channel,starttime,endtime,duration,gap_duration,uptime,availability
0,TA,M11A,,VHE,2007-02-15 00:00:09.999998,2007-02-24 23:59:59.999998,9 days 23:59:50,0 days,9 days 23:59:50,1.0
1,TA,M11A,,VHN,2007-02-15 00:00:09.999998,2007-02-24 23:59:59.999998,9 days 23:59:50,0 days,9 days 23:59:50,1.0
2,TA,M11A,,VHZ,2007-02-15 00:00:09.999998,2007-02-24 23:59:59.999998,9 days 23:59:50,0 days,9 days 23:59:50,1.0
3,TA,M14A,,VHE,2007-02-15 00:00:00.000003,2007-02-25 00:00:00.000003,10 days 00:00:00,0 days,10 days 00:00:00,1.0
4,TA,M14A,,VHN,2007-02-15 00:00:00.000003,2007-02-25 00:00:00.000003,10 days 00:00:00,0 days,10 days 00:00:00,1.0
5,TA,M14A,,VHZ,2007-02-15 00:00:00.000004,2007-02-25 00:00:00.000004,10 days 00:00:00,0 days,10 days 00:00:00,1.0


## Read index
`WaveBank` can return a dataframe of the the index with the `read_index` method, although in most cases this shouldn't be needed.

In [18]:
ta_bank.read_index().head()

,network,station,location,channel,starttime,endtime,sampling_period,path
0,TA,M11A,,VHN,2007-02-17 09:59:59.999998,2007-02-17 10:59:59.999998,0 days 00:00:10,TA/M11A/VHN/2007-02-17T10-00-00.mseed
1,TA,M14A,,VHN,2007-02-17 10:00:00.000003,2007-02-17 11:00:00.000003,0 days 00:00:10,TA/M11A/VHN/2007-02-17T10-00-00.mseed
2,TA,M11A,,VHN,2007-02-15 09:59:59.999998,2007-02-15 10:59:59.999998,0 days 00:00:10,TA/M11A/VHN/2007-02-15T10-00-00.mseed
3,TA,M14A,,VHN,2007-02-15 10:00:00.000003,2007-02-15 11:00:00.000003,0 days 00:00:10,TA/M11A/VHN/2007-02-15T10-00-00.mseed
4,TA,M11A,,VHN,2007-02-15 23:59:59.999998,2007-02-16 00:59:59.999998,0 days 00:00:10,TA/M11A/VHN/2007-02-16T00-00-00.mseed


## Similar Projects
`WaveBank` is a useful tool, but it may not be a good fit for every application. Check out the following items as well:

Obspy has a way to visualize availability of waveform data in a directory using [obspy-scan](https://docs.obspy.org/tutorial/code_snippets/visualize_data_availability_of_local_waveform_archive.html). If you prefer a graphical option to working with `DataFrame`s this might be for you.

Obspy also has [filesystem client](https://docs.obspy.org/master/packages/autogen/obspy.clients.filesystem.sds.Client.html#obspy.clients.filesystem.sds.Client) for working with SeisComP structured archives.

[IRIS](https://www.iris.edu/hq/) released a mini-seed indexing program called [mseedindex](https://github.com/iris-edu/mseedindex) which has an [ObsPy API](https://github.com/obspy/obspy/pull/2206).